In [1]:
import os
from os import getcwd
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from pathlib import Path
from sklearn.model_selection import GridSearchCV
import numpy as np

In [17]:
train = pd.read_parquet("/Users/camilleepitalon/Documents/DSB/04_python_for_data_science/03_projects/bike_counters-main/data/train.parquet")
test = pd.read_parquet("/Users/camilleepitalon/Documents/DSB/04_python_for_data_science/03_projects/bike_counters-main/data/train.parquet")
df_weather = pd.read_csv("submissions/starting_kit/external_data.csv", parse_dates=["date"])

In [18]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

In [19]:
def _encode_dates(X):
    X = X.copy()
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour
    return X.drop(columns=["date"])

In [20]:
def merge_external_data(X): 
    X_weather = df_weather[['date', 't', 'rr3']]
       #,'u', 'vv', 'ww', 'n', 'hbas', 'cl', 'tend', 'dd', 'ff', 'etat_sol', 'nnuage1', 'ctype1', 'hnuage3']]
    X["orig_index"] = np.arange(X.shape[0])
    X_merged = pd.merge_asof(X.sort_values("date"), X_weather.sort_values("date"), on='date')
    X_merged = X_merged.sort_values("orig_index")
    X_merged['t'] = X_merged['t'].fillna(0)
    X_merged['rr3'] = X_merged['rr3'].fillna(0)
    #X_merged['dd'] = X_merged['dd'].fillna(0)
    #X_merged['ff'] = X_merged['ff'].fillna(0)
    #X_merged['u'] = X_merged['u'].fillna(0)
    #X_merged['vv'] = X_merged['vv'].fillna(0)
    #X_merged['ww'] = X_merged['ww'].fillna(0)
    #X_merged['n'] = X_merged['n'].fillna(0)
    #X_merged['hbas'] = X_merged['hbas'].fillna(0)
    #X_merged['cl'] = X_merged['cl'].fillna(0)
    #X_merged['etat_sol'] = X_merged['etat_sol'].fillna(0)
    #X_merged['nnuage1'] = X_merged['nnuage1'].fillna(0)
    #X_merged['ctype1'] = X_merged['ctype1'].fillna(0)
    #X_merged['hnuage3'] = X_merged['hnuage3'].fillna(0)
    
    del X_merged["orig_index"]
    return X_merged

In [21]:
X_train_merged = merge_external_data(X_train)

In [22]:
X_good = _encode_dates(X_train_merged)

In [23]:
X_good = X_good.drop(columns=['counter_id', 'site_id', 'site_name', 'counter_installation_date', 'counter_technical_id', 'latitude', 'longitude'])
X_good

,counter_name,t,rr3,year,month,day,weekday,hour
0,152 boulevard du Montparnasse E-O,285.75,0.0,2020,9,1,1,1
30,152 boulevard du Montparnasse O-E,285.75,0.0,2020,9,1,1,1
31,18 quai de l'Hôtel de Ville NO-SE,285.75,0.0,2020,9,1,1,1
32,18 quai de l'Hôtel de Ville SE-NO,285.75,0.0,2020,9,1,1,1
33,20 Avenue de Clichy NO-SE,285.75,0.0,2020,9,1,1,1
...,...,...,...,...,...,...,...,...
455129,Totem 85 quai d'Austerlitz SE-NO,291.45,0.0,2021,8,9,0,23
455130,Totem Cours la Reine E-O,291.45,0.0,2021,8,9,0,23
455119,Totem Cours la Reine O-E,291.45,0.0,2021,8,9,0,23
455136,Voie Georges Pompidou NE-SO,291.45,0.0,2021,8,9,0,23


# Lightgbm

In [28]:
import lightgbm as ltb

# Call the merge_external_data function
merge_external = FunctionTransformer(merge_external_data, validate=False)

# Call the _encode_dates function to split the date column to several columns
encode_dates = FunctionTransformer(_encode_dates)

# Encode the final columns
#date_one_encoder = OneHotEncoder(handle_unknown="ignore")
#date_cols = ['year', 'month', 'day', 'weekday', 'hour']

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name"]

numeric_encoder = StandardScaler()
numeric_cols = ['year', 'month', 'day', 'weekday', 'hour', 't', 'rr3']

preprocessor = ColumnTransformer(
    [
        #("date", date_one_encoder, date_cols),
        ("cat", categorical_encoder, categorical_cols),
        ("numeric", numeric_encoder, numeric_cols) 
    ]
)
    
params = {
        'lambda_l1' : 4, 
        #'lambda_l2' : 9, 
        #'num_leaves' : 99,  
        #'max_depth' : 11,
        #'num_iterations' : 240,
        #'min_child_samples' : 6,
        #'max_bin' : 53, 
        #'n_estimators' : 1,
        #'subsample_for_bin' :  200,
        #'subsample' : 1,
        #'subsample_freq' : 1,
        #'alpha' : 20, 
        #'reg_alpha' : 0.306, 
        #'reg_lambda' : 0.306, 
        #'min_split_gain' : 0.5,
        #'min_child_weight' : 1,
        #'learning_rate' : 0.025
        }

regressor = ltb.LGBMRegressor(**params)

lgbm = make_pipeline(
    merge_external,
    encode_dates,
    preprocessor,
    regressor,
)

In [29]:
from sklearn.model_selection import GridSearchCV

lgbm.fit(X_train, y_train)

param_grid = {
    'lightgbmlambda_l1' : range(0, 11, 2),
    'lambda_l2' : range(, 17, 2)
}

grid = GridSearchCV(lgbm, param_grid, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

ValueError: Invalid parameter lambda_l2 for estimator Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function merge_external_data at 0x7f7d700d44c0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function _encode_dates at 0x7f7d700d4700>)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['counter_name']),
                                                 ('numeric', StandardScaler(),
                                                  ['year', 'month', 'day',
                                                   'weekday', 'hour', 't',
                                                   'rr3'])])),
                ('lgbmregressor', LGBMRegressor(lambda_l1=4))]). Check the list of available parameters with `estimator.get_params().k

In [118]:
print(grid.best_params_)
print(grid.best_score_)

{'lgbmregressor__n_jobs': 0}
-2.5091459679751997


In [107]:
print(grid.best_params_)
print(grid.best_score_)

{'lgbmregressor__n_estimators': 3}
-2.0207991809366477


In [ ]:
date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()

    
categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["site_name", "etat_sol", "nnuage1", "ctype1"]

numeric_encoder = StandardScaler()
numeric_cols = ['dd', 'ff', 't', 'u', 'vv', 'ww', 'n', 'hbas', "hnuage3", "tend", "cl", "rr3"]

preprocessor = ColumnTransformer(
        [
            ("date", "passthrough", date_cols),
            ("cat", categorical_encoder, categorical_cols),
            ("numeric", numeric_encoder, numeric_cols) 
        ]
)
    

regressor = ltb.LGBMRegressor(alpha = 20,num_leaves=100, n_estimators=200, lambda_l1=2, lambda_l2=2, max_depth=100)

pipe6 = make_pipeline(
    FunctionTransformer(merge_external_data, validate=False),
    date_encoder,
    preprocessor,
    regressor,
)

In [ ]:
scores = cross_val_score(
    pipe6,
    X_train,
    y_train,
    scoring="neg_root_mean_squared_error",
    error_score=np.nan,
)
print(f"RMSE: {-scores.mean():.3}± {(-scores).std():.3}")

#### 6. XG Boost

In [61]:
import xgboost as xgb

In [ ]:
date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()

    
categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["site_name", "etat_sol", "nnuage1", "ctype1"]

numeric_encoder = StandardScaler()
numeric_cols = ['dd', 'ff', 't', 'u', 'vv', 'ww', 'n', 'hbas', "hnuage3", "tend", "cl", "rr3"]

preprocessor = ColumnTransformer(
        [
            ("date", "passthrough", date_cols),
            ("cat", categorical_encoder, categorical_cols),
            ("numeric", numeric_encoder, numeric_cols) 
        ]
)


xg_reg = xgb.XGBRegressor(n_estimators = 91, colsample_bytree=0.3, max_depth=9, min_child_weight=4, max_delta_step=5)
    

pipe7 = make_pipeline(
    FunctionTransformer(merge_external_data, validate=False),
    date_encoder,
    preprocessor,
    xg_reg,
)

In [31]:
pipe7.get_params()

{'memory': None,
 'steps': [('functiontransformer-1',
   FunctionTransformer(func=<function merge_external_data at 0x131b335e0>)),
  ('functiontransformer-2',
   FunctionTransformer(func=<function _encode_dates at 0x12adabca0>)),
  ('columntransformer',
   ColumnTransformer(transformers=[('date', 'passthrough',
                                    ['year', 'month', 'day', 'weekday', 'hour']),
                                   ('cat', OneHotEncoder(handle_unknown='ignore'),
                                    ['site_name', 'etat_sol', 'nnuage1',
                                     'ctype1']),
                                   ('numeric', StandardScaler(),
                                    ['dd', 'ff', 't', 'u', 'vv', 'ww', 'n', 'hbas',
                                     'hnuage3', 'tend', 'cl', 'rr3'])])),
  ('xgbregressor',
   XGBRegressor(base_score=None, booster='gbtree', colsample_bylevel=None,
                colsample_bynode=None, colsample_bytree=0.3, gamma=None,
          

In [69]:
from sklearn.model_selection import GridSearchCV
import numpy as np

pipe7.fit(X_train, y_train)

param_grid = {
              'xgbregressor__subsample' : np.linspace(0,1,100)
             }

clf = GridSearchCV(pipe7, param_grid, cv=5, scoring='neg_mean_squared_error')
clf.fit(X_train, y_train)
print(clf.best_params_)

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

...

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-48-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

KeyboardInterrupt: 

In [32]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    pipe7,
    X_train,
    y_train,
    scoring="neg_root_mean_squared_error",
    error_score=np.nan,
)
print(f"RMSE: {-scores.mean():.3}± {(-scores).std():.3}")

<ipython-input-16-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-16-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["orig_index"] = np.arange(X.shape[0])
<ipython-input-16-9ae07d9db533>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

RMSE: 0.913± 0.178


## 3. PROPHET

In [ ]:
from prophet import Prophet

In [ ]:
pip install prophet

In [ ]:
date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()

    
categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["site_name", "etat_sol", "nnuage1", "ctype1"]

numeric_encoder = StandardScaler()
numeric_cols = ['dd', 'ff', 't', 'u', 'vv', 'ww', 'n', 'hbas', "hnuage3", "tend", "cl", "rr3"]

preprocessor = ColumnTransformer(
        [
            ("date", "passthrough", date_cols),
            ("cat", categorical_encoder, categorical_cols),
            ("numeric", numeric_encoder, numeric_cols) 
        ]
)
    
m = Prophet()
m.add_regressor('t')


pipe7 = make_pipeline(
    FunctionTransformer(merge_external_data, validate=False),
    date_encoder,
    preprocessor,
    regressor
)

In [ ]:
pipe7.fit(X_train)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    pipe7,
    X_train,
    y_train,
    scoring="neg_root_mean_squared_error",
    error_score=np.nan,
)
print(f"RMSE: {-scores.mean():.3}± {(-scores).std():.3}")

## 4. SARIMA


###### https://www.youtube.com/watch?v=z-uSBE8Pxwg

#### 1. Adfuller test to check the stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
result = adfuller(y_train) 

In [ ]:
print('ADF statistic: %f' %result[0])
print('p_value: %f' %result[1])
print('critical values:')

for key, value in result[4].items():
    print('\t%s: %3.f')

#### 2. ACF and PCF

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
fig, ax = plt.subplots(2, figsize=(12,6))
ax[0] = plot_acf(y_train, ax=ax[0], lags=20)
ax[1] = plot_pacf(y_train, ax=ax[1], lags=20)

#### 3. Trend, seasonality and residuals

In [ ]:
dftrain = train[['date', 'bike_count']]
dftrain = dftrain.groupby("date").sum()

In [ ]:
dftrain.head()

In [ ]:
period = len(dftrain)/8
print(period)

In [ ]:
import statsmodels.api as sm
ts_decomp = sm.tsa.seasonal_decompose(dftrain,model="additive", period=2000)
ts_decomp.plot()
plt.show()

#### 4. Parameters

In [ ]:
import statsmodels.api as sm

In [ ]:
model=sm.tsa.statespace.SARIMAX(train['bike_count'],order=(1,1,1),seasonal_order=(1,1,1,12))
results=model.fit()

In [ ]:
train['forecast']=results.predict(start=0,end=100,dynamic=True)
train[['bike_count','forecast']].plot(figsize=(12,8))

## 4. Consider time as a category

In [ ]:
X_train.head()

In [ ]:
X_train['hour'] = pd.to_numeric(X_train['hour'], downcast='float')
X_train['month'] = pd.to_numeric(X_train['month'], downcast='float')
X_train['weekday'] = pd.to_numeric(X_train['weekday'], downcast='float')
X_train['day'] = pd.to_numeric(X_train['day'], downcast='float')
X_train['year'] = pd.to_numeric(X_train['year'], downcast='float')
X_train.dtypes

In [ ]:
categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name", "ground_condition"]

numeric_encoder = StandardScaler()
numeric_cols = ["wind_direction", "nebulosity", "temperature", "humidity", "horizontal_visibility", "wind_velocity", "gusts_period", "rainfall_24"]

one_hot_linear_pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("categorical", OneHotEncoder, categorical_cols),
            ("numeric", numeric_encoder, numeric_cols),
            ("one_hot_time", OneHotEncoder, ["hour", "weekday", "month", "year"]),
        ],
        remainder=MinMaxScaler(),
    ),
    Ridge(alpha=0.5)
)

In [ ]:
scores = cross_val_score(
    one_hot_linear_pipeline,
    X_train,
    y_train,
    scoring="neg_root_mean_squared_error",
    error_score=np.nan,
)
print(f"RMSE: {-scores.mean():.3}± {(-scores).std():.3}")

## DE LA MERDE POUR LE MOMENT


#### 4. Periodic Splne feature

In [ ]:
X_train_merged = merge_external_data(X_train)
X_train_merged.columns
df = _encode_dates(X_train_merged[["date"]])
X_train_merged.drop(['date'], axis = 1, inplace = True)
X_train_merged = pd.concat([df, X_train_merged], axis=1)
X_train = X_train_merged
X_train.head()

In [ ]:
import pandas_profiling
X_train.profile_report()

In [ ]:
X_train.dtypes

In [ ]:
X_train['hour'] = pd.to_numeric(X_train['hour'], downcast='float')
X_train['month'] = pd.to_numeric(X_train['month'], downcast='float')
X_train['weekday'] = pd.to_numeric(X_train['weekday'], downcast='float')
X_train.dtypes

In [ ]:
from sklearn.preprocessing import SplineTransformer


def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )

In [ ]:
import matplotlib.pyplot as plt

hour_df = pd.DataFrame(
    np.linspace(0, 26, 1000).reshape(-1, 1),
    columns=["hour"],
)
splines = periodic_spline_transformer(24, n_splines=12).fit_transform(hour_df)
splines_df = pd.DataFrame(
    splines,
    columns=[f"spline_{i}" for i in range(splines.shape[1])],
)
pd.concat([hour_df, splines_df], axis="columns").plot(x="hour", cmap=plt.cm.tab20b)
_ = plt.title("Periodic spline-based encoding for the 'hour' feature")

In [ ]:
categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name", "ground_condition"]

numeric_encoder = StandardScaler()
numeric_cols = ["wind_direction", "nebulosity", "temperature", "humidity", "horizontal_visibility", "wind_velocity", "gusts_period", "rainfall_24"]

cyclic_spline_transformer = ColumnTransformer(
    transformers=[
        ("cat", categorical_encoder, categorical_cols),
        ("numeric", numeric_encoder, numeric_cols), 
        ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
        ("cyclic_weekday", periodic_spline_transformer(7, n_splines=3), ["weekday"]),
        ("cyclic_hour", periodic_spline_transformer(24, n_splines=12), ["hour"]),
    ],
    remainder=MinMaxScaler(),
)

cyclic_spline_linear_pipeline = make_pipeline(
    cyclic_spline_transformer,
    Ridge(alpha=0.5555),
)

scores = cross_val_score(
    cyclic_spline_linear_pipeline,
    X_train,
    y_train,
    scoring="neg_root_mean_squared_error",
    error_score=np.nan,
)

print(f"RMSE: {-scores.mean():.3}± {(-scores).std():.3}")